In [14]:
import sys
sys.path.append("..")
from shared_utilities.Betfair import BetfairClient
from azure.cosmos import CosmosClient
import shared_utilities.helpers as helpers
import time

In [15]:
# Do I read table first before upserting?

In [16]:
delay_api_key = helpers.read_json("../local.settings.json")["Values"]["betfair_delay_api_key"]
betfair_username = helpers.read_json("../local.settings.json")["Values"]["betfair_username"]
betfair_password = helpers.read_json("../local.settings.json")["Values"]["betfair_password"]
mma_id = 26420387
boxing_id = 6
horse_racing_id = 7

In [17]:
bf_client = BetfairClient(api_key=delay_api_key, username=betfair_username, password=betfair_password)

In [18]:
bf_client.listEvents(horse_racing_id)

[{'event': {'id': '33055352',
   'name': 'Catterick 27th Feb',
   'countryCode': 'GB',
   'timezone': 'Europe/London',
   'venue': 'Catterick',
   'openDate': '2024-02-27T14:20:00.000Z'},
  'marketCount': 21},
 {'event': {'id': '32996854',
   'name': 'Randwick (AUS) 20th April',
   'countryCode': 'AU',
   'timezone': 'Australia/Sydney',
   'openDate': '2024-04-20T05:00:00.000Z'},
  'marketCount': 2},
 {'event': {'id': '33055354',
   'name': 'Southwell 27th Feb',
   'countryCode': 'GB',
   'timezone': 'Europe/London',
   'venue': 'Southwell',
   'openDate': '2024-02-27T16:28:00.000Z'},
  'marketCount': 53},
 {'event': {'id': '33060734',
   'name': 'Southwell (F/C) 27th Feb',
   'countryCode': 'GB',
   'timezone': 'Europe/London',
   'venue': 'Southwell',
   'openDate': '2024-02-27T16:28:00.000Z'},
  'marketCount': 2},
 {'event': {'id': '33060464',
   'name': 'Angers (FRA) 28th Feb',
   'countryCode': 'FR',
   'timezone': 'Europe/London',
   'venue': 'Angers',
   'openDate': '2024-02-28T

In [19]:
# test = bf_client.getMarkets(event_ids=[32998380,32997720],inplay_markets="False")
# test
# id_list = []
# for i in test[0]["result"]:
#     id_list.append(i["event"]["id"])
# id_list

In [20]:
bf_client.getMarketIdMarkets(1.225327448, False)

[{'jsonrpc': '2.0', 'result': [], 'id': 1}]

In [22]:
bf_client.listEvents(mma_id)

[{'event': {'id': '33008275',
   'name': 'Matt Schnell v Steve Erceg',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2024-03-02T21:15:00.000Z'},
  'marketCount': 2},
 {'event': {'id': '33008274',
   'name': 'Julia Polastri v Josefine Knutsson',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2024-03-02T18:10:00.000Z'},
  'marketCount': 2},
 {'event': {'id': '33037470',
   'name': 'Christian Leroy Duncan v Claudio Ribeiro',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2024-03-02T19:50:00.000Z'},
  'marketCount': 2},
 {'event': {'id': '33008273',
   'name': 'Loik Radzhabov v Abdul-Kareem Al-Selwady',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2024-03-02T19:00:00.000Z'},
  'marketCount': 1},
 {'event': {'id': '33008272',
   'name': 'Eryk Anders v Jamie Pickett',
   'countryCode': 'GB',
   'timezone': 'GMT',
   'openDate': '2024-03-02T22:15:00.000Z'},
  'marketCount': 2},
 {'event': {'id': '32943999',
   'name': 'Tai Tuivasa

In [24]:
print(bf_client.getMarkets([33008275],True))
print(bf_client.getMarkets([33008275],True))

[{'jsonrpc': '2.0', 'result': [], 'id': 1}]
[{'jsonrpc': '2.0', 'result': [], 'id': 1}]


In [27]:
bf_client.eventStartedChecker([32988787])


Event has not started yet


In [9]:
from datetime import datetime

In [10]:
# # if timezone == GMT
# atime= "2024-02-04T02:30:00.000Z"
# datetime.strptime(atime, '%Y-%m-%dT%H:%M:%S.%fZ')
helpers.find_next_weekend()

('2024-02-17', '2024-02-18', '2024-02-19')

In [11]:
conn_string = helpers.read_json("../local.settings.json")["Values"]["cosmosdb_deets"]
storageClient = CosmosClient.from_connection_string(conn_str=conn_string)
database = storageClient.get_database_client("fightstore")
fc_mma_cards = database.get_container_client("fc_mma_cards")

next_saturday, next_sunday, next_monday = helpers.find_next_weekend()

# select next weekend card order by 


queryText = f"""
SELECT  c.title
       ,c.date
       ,c.id
       ,c.cardDate
       ,c.link
       ,cx.betfair_event_id
       ,cx.fight_name
       ,cx.betfair_open_date
FROM   c
JOIN   cx IN c.fights
WHERE cx.betfair_open_date BETWEEN '{next_saturday}' AND '{next_monday}'
"""

results = fc_mma_cards.query_items(query=queryText, enable_cross_partition_query=True)

In [12]:
next_sunday

'2024-02-18'

In [13]:
items = [item for item in results]

In [14]:
items

[{'title': 'UFC 298',
  'date': 'Saturday, February 17,  6:30 PM ET',
  'id': '7840535991427417239',
  'cardDate': '2024-02-17',
  'link': 'https://www.tapology.com/fightcenter/events/107060-ufc-298',
  'betfair_event_id': '32834431',
  'fight_name': 'Alexander Volkanovski v Ilia Topuria',
  'betfair_open_date': '2024-02-18T05:15:00.000Z'},
 {'title': 'UFC 298',
  'date': 'Saturday, February 17,  6:30 PM ET',
  'id': '7840535991427417239',
  'cardDate': '2024-02-17',
  'link': 'https://www.tapology.com/fightcenter/events/107060-ufc-298',
  'betfair_event_id': '32894099',
  'fight_name': 'Robert Whittaker v Paulo Costa',
  'betfair_open_date': '2024-02-18T04:45:00.000Z'},
 {'title': 'UFC 298',
  'date': 'Saturday, February 17,  6:30 PM ET',
  'id': '7840535991427417239',
  'cardDate': '2024-02-17',
  'link': 'https://www.tapology.com/fightcenter/events/107060-ufc-298',
  'betfair_event_id': '32894098',
  'fight_name': 'Geoff Neal v Ian Machado Garry',
  'betfair_open_date': '2024-02-18T

In [15]:
# items.sort(key=lambda item:item['betfair_open_date'], reverse=True) # can't do this because it changes the index for writing to cosmos

In [16]:
int(time.time())

1707953023

In [17]:
t_end = time.time() + (8*60)
while time.time() < t_end:
    for idx, item in  enumerate(items):
        if bf_client.eventStartedChecker(item["betfair_event_id"]) == True:
            operations =[{ "op": "add", "path": "/fights/"+str(idx)+"/start_time", "value": int(time.time()) }]
            response = fc_mma_cards.patch_item(item=item["id"], partition_key=item["link"], patch_operations=operations)
    
    
# change this so that it groups the API calls together? rather than doing a single api call for each event..?
    

Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has not started yet
Event has no

KeyboardInterrupt: 